In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

## read json

In [0]:
df = spark.read.format("json")\
      .option("inferSchema", "true")\
      .option("multiLine", "true")\
      .load("/Volumes/streaming/stream/source/day1.json")

display(df)

In [0]:
df1 = df.select('order_id','timestamp','customer.customer_id','customer.name','customer.address.city','customer.address.country','payment.method','payment.transaction_id','items','metadata')

display(df1)

In [0]:
df2 = df1.withColumn('items',explode_outer('items'))\
    .withColumn('metadata',explode_outer('metadata'))


display(df2)



In [0]:
df3 = df2.select('*','items.item_id','items.product_name','items.quantity','items.price','metadata.key','metadata.value').drop('items','metadata')

display(df3)

In [0]:
df3.write.format("delta")\
    .mode("append")\
    .save("/Volumes/streaming/stream/sink")

In [0]:
%sql
select * from delta.`/Volumes/streaming/stream/sink`

## json read

In [0]:
df = spark.read.format("json")\
    .option("inferSchema","true")\
    .option("multiline","true")\
    .load("/Volumes/streaming/stream/source/example.json")

display(df)

In [0]:
df1 = df.select('rental_id','rented_on','return_due','status','member.member_id','member.full_name','member.contact.phone','member.membership_type','fines.accumulated','fines.currency','books','notes')

display(df1)

In [0]:
df2 = df1.withColumn("books",explode_outer("books"))\
    .withColumn("notes",explode_outer("notes"))

display(df2)

In [0]:
df3 = df2.select('*','books.book_id','books.title','books.author','books.rental_days','notes.tag','notes.description').drop('books','notes')

display(df3)

In [0]:
df3.write.format("delta")\
    .mode("overwrite")\
    .save("/Volumes/streaming/stream/sink1")

In [0]:
%sql
select * from delta.`/Volumes/streaming/stream/sink1`